In [1]:
import pandas as pd
import os
from simpledbf import Dbf5
import plotly.express as px
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mtick
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [2]:
# .dbf to df
data_to_load = os.path.join("resources", "S_USA.Fire_Occurrence_FIRESTAT_YRLY","S_USA.Fire_Occurrence_FIRESTAT_YRLY.dbf")
dbf = Dbf5(data_to_load)
df = dbf.to_dataframe()

In [3]:
df.head()

,FIRE_NAME,LOCAL_FIRE,LOCATION,TOWNSHIP,RANGE,SECTION,SUB_SECTIO,PRINCIPAL_,REPORT_UNI,REPORT_U_1,...,COVER_CLAS,SLOPE,ASPECT,ELEVATION,LOCAL_TIME,RECORD_ENT,SUBMITTED_,APPROVED_D,CREATED_DA,LAST_MODIF
0,SPOONER,NaN,NaN,0140N,0180E,02,NE,MT. DIABLO,0519,Lake Tahoe Basin Management Unit,...,23 - Moderate Chamise and chaparral,10,3 East,7000,America/Los_Angeles,1998-09-26,NaN,NaN,2005-02-04,2015-12-09
1,SPRING,NaN,NaN,0210S,0570E,34,SE,MT. DIABLO,0417,Humboldt-Toiyabe National Forest,...,33 - Pinyon-Juniper,15,3 East,5500,America/Los_Angeles,1998-10-20,NaN,NaN,2005-02-04,2015-12-09
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MT. DIABLO,0409,Humboldt National Forest,...,-,35,5 South,8000,America/Los_Angeles,1987-12-14,NaN,NaN,2005-02-04,2015-12-09
3,SEVENTEEN,NaN,NaN,0180S,0540E,17,NWSE,MT. DIABLO,0417,Humboldt-Toiyabe National Forest,...,33 - Pinyon-Juniper,10,7 West,6700,America/Los_Angeles,2002-12-12,NaN,NaN,2005-02-04,2015-12-09
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MT. DIABLO,0417,Humboldt-Toiyabe National Forest,...,-,35,2 Northeast,8000,America/Los_Angeles,1986-11-24,NaN,NaN,2005-02-04,2015-12-09


In [4]:
# Cleaning Data
df= df.sort_values(by = "FIRE_OUT", ascending = False).iloc[:200000]
pd.to_datetime(df["FIRE_OUT"])
df.FIRE_NAME = df.FIRE_NAME.fillna('')
df["FIRE_OUT"].head()
df = df.dropna(axis = 1)
df = df.drop(["ADMIN_UN_1","OWNERSHIP_",
                  "NFDRS_FUEL","PROTECTING","AGENCY_ACR","OTHER_ACRE",
                 "OTHER_AC_1", "PRESCRIB_1","REPORT_UNI","REPORT_U_1",
                  "ADMIN_UNIT","LAT_DEG","LAT_MIN","LAT_SEC",
                 "LONG_DEG","LONG_MIN","LONG_SEC","FIRE_SIZE_","WUI_ACRES",
                  "REP_WX_STA","STATION_TY","OTHER_FUEL"], axis = 1)
df["STATE_NAME"] = df["STATE_NAME"].str[:2]
df.STATISTICA =  df.STATISTICA.str.extract(r'(( )\w+)$').iloc[: , :-1]
df.FIRE_INTEN = df.FIRE_INTEN.replace(regex={r'\d\s-\s': ''})
df.FIRE_DETEC = df.FIRE_DETEC.replace(regex={r'(\d)?\d\s-\s(\s)?': ''})
df.COVER_CLAS = df.COVER_CLAS.replace(regex={r'\d\d\s-\s': ''})


In [5]:
df.head()

,FIRE_NAME,FIRE_NUMBE,STATE_NAME,POO_LATITU,POO_LONGIT,DISCOVER_Y,DISCOVERY,FIRE_DETEC,STATISTICA,FIRE_OUT,TOTAL_ACRE,FIRE_INTEN,WIND_SPEED,COVER_CLAS,SLOPE,ELEVATION,CREATED_DA,LAST_MODIF
167115,Shanklin,015,KY,36.814692,-87.996194,2020,2020-12-14,Agency Patrol,Arson,2020-12-31,1.00,Flame Length 0-2',0,Bottom hardwood,10,750,2021-04-01,2021-04-01
115627,Present,071,CA,33.312944,-116.858861,2020,2020-12-25,Neighbor,Campfire,2020-12-25,0.01,Flame Length 0-2',0,Medm slash-long needle 10+yr,7,5579,2020-12-29,2020-12-29
58731,Brown,070,CA,32.860139,-116.693583,2020,2020-12-23,Resident,Campfire,2020-12-23,0.01,Flame Length 0-2',0,Perennial grass,0,2520,2020-12-24,2020-12-28
115628,Bond,068,CA,33.744700,-117.636560,2020,2020-12-03,Resident,Miscellaneous,2020-12-21,6687.39,Flame Length >12',0,Annual grass,45,1333,2020-12-28,2020-12-31
57379,COUNTY,046,CA,41.882660,-121.324600,2020,2020-08-12,Coop Agency Patrol,Miscellaneous,2020-12-17,1.00,Flame Length 0-2',0,Annual grass,0,4400,2020-12-31,2021-01-29


In [6]:
print(df.FIRE_DETEC.unique())

['Agency Patrol' 'Neighbor' 'Resident' 'Coop Agency Patrol' 'Passerby'
 'Cooperator or Coop employ' 'Visitor in area' 'Agency Patrol aircraft'
 'Aircraft in vicinity' 'Agency Lookout' 'Other Agency Employee'
 'Contractor personnel' 'Others' 'Tenant, permittee'
 'Aerial det, type unkwn/NR' 'Planned Coop' 'On duty agency employee'
 'Roving non-fire patrol' 'Non-Occupant, type unk/NR'
 'Coop Agency fire guard' 'Detector known, unclassed'
 'Aerial Infrared detection' 'Infrared Detect' '-'
 'Aircraft on regular route' 'Non-Agency Lookout'
 'Occupant, type unknown/NR' 'Agency Permittee' 'Coop Agency Lookout'
 'Other Air Obs' 'Coop patrol aircraft' 'Roving ptrl, not class ab'
 'Unclassified detection' 'Satellite detection' 'Building/Property guard'
 'Rover, type unknown/NR' 'Auto detect systm w/alarm'
 'Fire detected, no alarm' 'Auto det, type unknown/NR' 'Agency Air Obs'
 'Rpt Agency roving patrol' 'Watchman, type unknown/NR'
 'Watchman, not class above' 'Occupant, not class above'
 'Rpt Age

In [12]:
df = df.rename(columns={"POO_LATITU": "lat", "POO_LONGIT": "lng"}, errors="raise")
#df.to_csv("analysis.csv")
df.head()

,FIRE_NAME,FIRE_NUMBE,STATE_NAME,lat,lng,DISCOVER_Y,DISCOVERY,FIRE_DETEC,STATISTICA,FIRE_OUT,TOTAL_ACRE,FIRE_INTEN,WIND_SPEED,COVER_CLAS,SLOPE,ELEVATION,CREATED_DA,LAST_MODIF
167115,Shanklin,015,KY,36.814692,-87.996194,2020,2020-12-14,Agency Patrol,Arson,2020-12-31,1.00,Flame Length 0-2',0,Bottom hardwood,10,750,2021-04-01,2021-04-01
115627,Present,071,CA,33.312944,-116.858861,2020,2020-12-25,Neighbor,Campfire,2020-12-25,0.01,Flame Length 0-2',0,Medm slash-long needle 10+yr,7,5579,2020-12-29,2020-12-29
58731,Brown,070,CA,32.860139,-116.693583,2020,2020-12-23,Resident,Campfire,2020-12-23,0.01,Flame Length 0-2',0,Perennial grass,0,2520,2020-12-24,2020-12-28
115628,Bond,068,CA,33.744700,-117.636560,2020,2020-12-03,Resident,Miscellaneous,2020-12-21,6687.39,Flame Length >12',0,Annual grass,45,1333,2020-12-28,2020-12-31
57379,COUNTY,046,CA,41.882660,-121.324600,2020,2020-08-12,Coop Agency Patrol,Miscellaneous,2020-12-17,1.00,Flame Length 0-2',0,Annual grass,0,4400,2020-12-31,2021-01-29


In [14]:
# columns used for constructing geojson object
features = df.apply(
    lambda row: Feature(geometry=Point((float(row['lng']), float(row['lat'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = df.drop(['lat', 'lng'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

In [15]:
geoFile = df.to_json(orient='records')
with open('map.json','w') as f:
    f.write(geoFile)